In [1]:
# Connect to google drive to fetch preprocessed data of Kaggle Dataset
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import cv2
import gc
from sklearn.model_selection import KFold
from google.colab.patches import cv2_imshow

In [12]:
# Load Correctly Masked Faces
correctly_features=np.load("/content/drive/MyDrive/MSProjectData/Dataset2/Correct/correct_features.npz")
# Load Incorrectly Masked Faces
incorrect_features=np.load("/content/drive/MyDrive/MSProjectData/Dataset2/Incorrect/incorrect_features.npz")

In [13]:
# Extract Correctly Masked Features from npz 
correctly_masked=correctly_features['arr_0']
correctly_masked=correctly_masked/255
# Extract Incorrectly Masked Features from npz
incorrectly_masked=incorrect_features['arr_0']
incorrectly_masked=incorrectly_masked/255

In [14]:
del correctly_features
del incorrect_features

In [15]:
print(correctly_masked.shape,incorrectly_masked.shape)

(2287, 16, 16, 1) (550, 16, 16, 1)


In [17]:
# Preparing Training And Testing Data for 5-fold cross validation
trainingCorrectlyMasked=[]
testingCorrectlyMasked=[]
trainingIncorrectlyMasked=[]
testingIncorrectlyMasked=[]
kfold=KFold(n_splits=5,shuffle=True)
for train,test in kfold.split(range(len(correctly_masked))):
  trainingCorrectlyMasked.append(correctly_masked[train])
  testingCorrectlyMasked.append(correctly_masked[test])
for train,test in kfold.split(range(len(incorrectly_masked))):
  trainingIncorrectlyMasked.append(incorrectly_masked[train])
  testingIncorrectlyMasked.append(incorrectly_masked[test])
trainingCorrectlyMasked=np.asarray(trainingCorrectlyMasked)
trainingIncorrectlyMasked=np.asarray(trainingIncorrectlyMasked)
testingCorrectlyMasked=np.asarray(testingCorrectlyMasked)
testingIncorrectlyMasked=np.asarray(testingIncorrectlyMasked)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [18]:
print(trainingCorrectlyMasked.shape,trainingIncorrectlyMasked.shape,testingCorrectlyMasked.shape,testingIncorrectlyMasked.shape)

(5,) (5, 440, 16, 16, 1) (5,) (5, 110, 16, 16, 1)


In [19]:
# Import Tesortflow and Keras libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv1D,Conv2D,Flatten,MaxPool2D
from tensorflow.keras.callbacks import EarlyStopping

In [21]:
accuracy=[]
correctlyAccuracy=[]
incorrectlyAccuracy=[]
for i in range(0,5):
  # Preparing Training And Testing Set
  x_train=np.concatenate((trainingCorrectlyMasked[i],trainingIncorrectlyMasked[i]),axis=0)
  x_test=np.concatenate((testingCorrectlyMasked[i],testingIncorrectlyMasked[i]),axis=0)
  y_train=np.concatenate((np.zeros(len(trainingCorrectlyMasked[i])),np.ones(len(trainingIncorrectlyMasked[i]))),axis=0)
  y_test=np.concatenate((np.zeros(len(testingCorrectlyMasked[i])),np.ones(len(testingIncorrectlyMasked[i]))),axis=0)
  print(x_train.shape,y_train.shape,x_test.shape,y_test.shape)
  # CNN Model
  model=Sequential()
  model.add(Conv2D(filters=128,kernel_size=(5,5),activation='relu',input_shape=x_train[0].shape))
  model.add(MaxPool2D(pool_size=(2,2)))
  model.add(Conv2D(filters=32,kernel_size=(3,3),activation='relu'))
  model.add(MaxPool2D(pool_size=(2,2)))
  model.add(Flatten())
  model.add(Dense(units=128,activation='relu'))
  model.add(Dense(units=32,activation='relu'))
  model.add(Dense(units=1,activation='sigmoid'))
  model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
  model.summary()
  class MyClass(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
      # Stop the model if accuracy reaches 99.5%
      if logs.get('accuracy') > 0.995:
        self.model.stop_training=True
  model.fit(x_train,y_train,epochs=500,batch_size=128,shuffle=True,callbacks=[MyClass()])
  accuracy.append(model.evaluate(x_test,y_test)[1])
  correctlyAccuracy.append(model.evaluate(testingCorrectlyMasked[i],np.zeros(len(testingCorrectlyMasked[i])))[1])
  incorrectlyAccuracy.append(model.evaluate(testingIncorrectlyMasked[i],np.ones(len(testingIncorrectlyMasked[i])))[1])

(2269, 16, 16, 1) (2269,) (568, 16, 16, 1) (568,)
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 12, 12, 128)       3328      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 128)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 4, 32)          36896     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 2, 2, 32)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               16512     
_________________________________________________________________
dens

In [22]:
print("Average Accuracy:",sum(accuracy)/len(accuracy)*100)
print("\n")
print("Average Accuracy for Correctly Masked Faces:",sum(correctlyAccuracy)/len(correctlyAccuracy)*100)
print("Average Accuract for Incorrectly Masked only Chin Covered:",sum(incorrectlyAccuracy)/len(incorrectlyAccuracy)*100)


Average Accuracy: 91.85731530189514


Average Accuracy for Correctly Masked Faces: 95.58225750923157
Average Accuract for Incorrectly Masked only Chin Covered: 76.36363625526428


In [23]:
model.save("/content/drive/MyDrive/MSProjectData/Dataset2/KaggleModel.h5")